<a href="https://colab.research.google.com/github/moqri/deep_cell/blob/master/starter_code/blood_cells.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#install required packages
!pip install scanpy

In [0]:
# Download the data 
!mkdir kidney
!wget -O kidney/exprMatrix.tsv.gz http://cells.ucsc.edu/kidney-atlas/Mature_Full/exprMatrix.tsv.gz 
!wget -O kidney/meta.tsv http://cells.ucsc.edu/kidney-atlas/Mature_Full/meta.tsv

mkdir: cannot create directory ‘kidney’: File exists
--2020-01-18 15:42:01--  http://cells.ucsc.edu/kidney-atlas/Mature_Full/exprMatrix.tsv.gz
Resolving cells.ucsc.edu (cells.ucsc.edu)... 128.114.119.132, 128.114.119.131
Connecting to cells.ucsc.edu (cells.ucsc.edu)|128.114.119.132|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 175277031 (167M) [application/x-gzip]
Saving to: ‘kidney/exprMatrix.tsv.gz’

kidney/exprMatrix.t 100%[===================>] 167.16M  21.0MB/s    in 9.0s    

2020-01-18 15:42:11 (18.5 MB/s) - ‘kidney/exprMatrix.tsv.gz’ saved [175277031/175277031]

--2020-01-18 15:42:12--  http://cells.ucsc.edu/kidney-atlas/Mature_Full/meta.tsv
Resolving cells.ucsc.edu (cells.ucsc.edu)... 128.114.119.132, 128.114.119.131
Connecting to cells.ucsc.edu (cells.ucsc.edu)|128.114.119.132|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3906356 (3.7M) [text/tab-separated-values]
Saving to: ‘kidney/meta.tsv’

kidney/meta.tsv     100%[====

In [0]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm
import itertools
import numpy as np
import scanpy as sc
import pandas as pd

In [0]:
folder='kidney'
ad = sc.read_text(folder+"/exprMatrix.tsv.gz")
meta = pd.read_csv(folder+"/meta.tsv", sep="\t")
ad.var = meta
ad.data.shape

AnnData expects string indices for some functionality, but your first two indices are: RangeIndex(start=0, stop=2, step=1). 
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Use X instead of data, data will be removed in the future.
  """


(33694, 40268)

In [0]:
expression_df=pd.DataFrame(ad.data).transpose()
expression_df.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Use X instead of data, data will be removed in the future.
  """Entry point for launching an IPython kernel.


(40268, 33694)

In [0]:
expression_df.index.rename('Cell_ID',inplace=1)
expression_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,33654,33655,33656,33657,33658,33659,33660,33661,33662,33663,33664,33665,33666,33667,33668,33669,33670,33671,33672,33673,33674,33675,33676,33677,33678,33679,33680,33681,33682,33683,33684,33685,33686,33687,33688,33689,33690,33691,33692,33693
Cell_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.968848,2.549508,0.0,0.000000,2.863407,2.345706,0.0,2.089452,0.0,0.0,2.345706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.751524,0.000000,0.0,0.000000,0.000000,0.000000,0.0,4.432036,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.797596,3.915623,0.0,4.018989,0.000000,3.800328,0.0,4.418446,0.0,0.0,3.669986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.613472,4.008724,0.0,4.008724,4.571343,0.000000,0.0,0.000000,0.0,0.0,3.844965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
cell_types=meta['celltype']
cell_df=pd.DataFrame(cell_types)
cell_df.index.rename('Cell_ID',inplace=1)
cell_df.columns=['Cell_Type']
cell_df.head()

,Cell_Type
Cell_ID,
0,Thick ascending limb of Loop of Henle
1,Peritubular capillary endothelium 2
2,Proximal tubule
3,NK cell
4,Proximal tubule


In [0]:
cell_df['Cell_Type'].value_counts()

Proximal tubule                          27497
CD4 T cell                                2435
NK cell                                   1322
CD8 T cell                                 991
B cell                                     975
NKT cell                                   733
Thick ascending limb of Loop of Henle      675
Peritubular capillary endothelium 1        666
Glomerular endothelium                     622
MNP-a/classical monocyte derived           555
Indistinct intercalated cell               429
MNP-b/non-classical monocyte derived       375
Proliferating Proximal Tubule              348
Ascending vasa recta endothelium           300
Type A intercalated cell                   275
Epithelial progenitor cell                 238
MNP-c/dendritic cell                       220
Descending vasa recta endothelium          192
Peritubular capillary endothelium 2        185
Connecting tubule                          157
Distinct proximal tubule 2                 151
Pelvic epithe

In [0]:
expression_df=expression_df.loc[cell_df.index]
cell_df['Cell_Type'] = pd.Categorical(cell_df['Cell_Type'])
cell_df['Cell_Type'] = cell_df['Cell_Type'].cat.codes

In [0]:
expression_array=expression_df.values
expression_array=expression_array[:,~np.all(expression_array == 0, axis=0)] # remove all zero columns
expression_array=np.log(expression_array+1)
cell_array=cell_df.values

In [0]:
number_of_genes_top_keep=1000
SelectKBest(chi2, k=number_of_genes_top_keep).fit_transform(expression_array, cell_array)
expression_array=SelectKBest(chi2, k=number_of_genes_top_keep).fit_transform(expression_array, cell_array)
expression_array=np.round(expression_array) 

In [0]:
np.random.seed(1)
X_train, X_test, y_train, y_test = train_test_split(
    expression_array, cell_array, test_size=0.33, random_state=1)
clf = svm.SVC(decision_function_shape='ovo')
y_predict=clf.fit(X_train, y_train)
y_predict=clf.predict(X_test)
accuracy_score(y_test, y_predict)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9689969147415155

In [0]:
predictions=pd.DataFrame(data={'label':list(itertools.chain(*y_test)),'predicted':list(y_predict)})
predictions.head()

,label,predicted
0,28,28
1,28,28
2,2,2
3,28,28
4,28,28


In [0]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
tf.random.set_seed(1)

In [0]:
dataset=tf.data.Dataset.from_tensor_slices((expression_array,cell_array ))
dataset

<TensorSliceDataset shapes: ((1000,), (1,)), types: (tf.float32, tf.int8)>

In [0]:
for feat, targ in dataset.take(1):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0.
 0. 2. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0.
 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 1

In [0]:
train_dataset = dataset.shuffle(len(cell_df)).batch(1)
cell_type_count=len(cell_df['Cell_Type'].value_counts())

In [0]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(cell_type_count, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [0]:
model = get_compiled_model()
model.fit(X_train, y_train, epochs=10)
model.evaluate(X_test,  y_test, verbose=2)

Train on 26979 samples
Epoch 1/10
26979/26979 [==============================] - 2s 78us/sample - loss: 0.2514 - accuracy: 0.9401
Epoch 2/10
26979/26979 [==============================] - 2s 66us/sample - loss: 0.0942 - accuracy: 0.9746
Epoch 3/10
26979/26979 [==============================] - 2s 65us/sample - loss: 0.0716 - accuracy: 0.9799
Epoch 4/10
26979/26979 [==============================] - 2s 64us/sample - loss: 0.0563 - accuracy: 0.9851
Epoch 5/10
26979/26979 [==============================] - 2s 64us/sample - loss: 0.0464 - accuracy: 0.9877
Epoch 6/10
26979/26979 [==============================] - 2s 69us/sample - loss: 0.0387 - accuracy: 0.9891
Epoch 7/10
26979/26979 [==============================] - 2s 66us/sample - loss: 0.0302 - accuracy: 0.9920
Epoch 8/10
26979/26979 [==============================] - 2s 66us/sample - loss: 0.0256 - accuracy: 0.9930
Epoch 9/10
26979/26979 [==============================] - 2s 66us/sample - loss: 0.0176 - accuracy: 0.9956
Epoch 10/10
26

[0.23265882869982485, 0.9606441]